In [1]:
import pandas as pd
import numpy as np
import joblib
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

In [2]:
df = pd.read_csv("../Data/AQI and Lat Long of Countries_cleaned.csv")
df

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,Russian Federation,Praskoveya,51,2,1,1,36,1,0,1,51,2,44.7444,44.2031
1,Brazil,Presidente Dutra,41,1,1,1,5,1,1,1,41,1,-5.2900,-44.4900
2,Brazil,Presidente Dutra,41,1,1,1,5,1,1,1,41,1,-11.2958,-41.9869
3,Italy,Priolo Gargallo,66,2,1,1,39,1,2,1,66,2,37.1667,15.1833
4,Poland,Przasnysz,34,1,1,1,34,1,0,1,20,1,53.0167,20.8833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16690,United States of America,Highland Springs,54,2,1,1,34,1,5,1,54,2,37.5516,-77.3285
16691,Slovakia,Martin,71,2,1,1,39,1,1,1,71,2,49.0650,18.9219
16692,Slovakia,Martin,71,2,1,1,39,1,1,1,71,2,36.3385,-88.8513
16693,France,Sceaux,50,1,1,1,20,1,5,1,50,1,48.7786,2.2906


In [3]:
# Define features & targets
FEATURES = ['Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value', 'CO AQI Value',  'lat']
TARGET_AQI_VALUE = 'AQI Value'
TARGET_AQI_CATEGORY = 'AQI Category'

In [5]:
# Selecting features & target
X = df[FEATURES]
y = df[TARGET_AQI_VALUE]# Split data
X = df[FEATURES]
y_value = df[TARGET_AQI_VALUE]
y_category = df[TARGET_AQI_CATEGORY]
X_train, X_test, y_train_value, y_test_value = train_test_split(X, y_value, test_size=0.2, random_state=42)
X_train_cat, X_test_cat, y_train_category, y_test_category = train_test_split(X, y_category, test_size=0.2, random_state=42)

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Initialize and train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [8]:
# Predictions
y_pred = xgb_model.predict(X_test)

In [9]:
# Evaluation
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 0.5356537583846109
Root Mean Squared Error: 5.468069737865821


In [10]:
r2 = r2_score(y_test, y_pred)  # R² Score
r2

0.9819719791412354

In [11]:
# Save model
model_path = "../Model/AQI_XGB_Model.pkl"
joblib.dump(xgb_model, model_path)
print(f"Model saved to {model_path}")

Model saved to ../Model/AQI_XGB_Model.pkl
